# practise the cnn

In [39]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline

mnist = input_data.read_data_sets('../data/', one_hot=True)

Extracting ../data/train-images-idx3-ubyte.gz
Extracting ../data/train-labels-idx1-ubyte.gz
Extracting ../data/t10k-images-idx3-ubyte.gz
Extracting ../data/t10k-labels-idx1-ubyte.gz


In [56]:
# build the graph: conv1, fc1, softmax
n_input = 28 * 28 * 1
n_classes = 10
x = tf.placeholder(tf.float32, shape=[None, n_input])
y = tf.placeholder(tf.float32, shape=[None, n_classes])
x_ = tf.reshape(x, shape=[-1, 28, 28, 1])
keep_prob = tf.placeholder(tf.float32)

# conv1 layer
conv1_n = 32
filter_size = 3
conv1 = {"weight": tf.Variable(tf.truncated_normal([filter_size, filter_size, 1, conv1_n], stddev=0.1)), 
        "bias": tf.Variable(tf.truncated_normal([1, conv1_n], stddev=0.1))}

conv1_z = tf.nn.conv2d(x_, conv1["weight"], strides=[1, 1, 1, 1], padding="SAME")
conv1_a = tf.nn.relu(conv1_z + conv1["bias"])
pool1_a = tf.nn.max_pool(conv1_a, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
print "conv1 layer with max pool  build done, output [, 14, 14, 32]"

# conv2 layer
conv2_n = 64
filter_size = 3
conv2 = {"weight": tf.Variable(tf.truncated_normal([filter_size, filter_size, conv1_n, conv2_n], stddev=0.1)), 
    "bias": tf.Variable(tf.truncated_normal([1, conv2_n], stddev=0.1))}

conv2_z = tf.nn.conv2d(pool1_a, conv2["weight"], strides=[1, 1, 1, 1], padding="SAME")
conv2_a = tf.nn.relu(conv2_z + conv2["bias"])
pool2_a = tf.nn.max_pool(conv2_a, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
print "conv2 layer with max pool  build done, output [, 7, 7, 64]"
width = 7
nodes = conv2_n
outting = pool2_a

# dropout layer
outting = tf.nn.dropout(outting, keep_prob)

# conv3 layer
conv3_n = 64 * 2
filter_size = 3
conv3 = {"weight": tf.Variable(tf.truncated_normal([filter_size, filter_size, conv2_n, conv3_n], stddev=0.1)), 
    "bias": tf.Variable(tf.truncated_normal([1, conv3_n], stddev=0.1))}

conv3_z = tf.nn.conv2d(pool2_a, conv3["weight"], strides=[1, 1, 1, 1], padding="SAME")
conv3_a = tf.nn.relu(conv3_z + conv3["bias"])
pool3_a = tf.nn.max_pool(conv3_a, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
print "conv3 layer with max pool  build done, output [, 4, 4, 128]"
width = 4
nodes = conv3_n
outting = pool3_a

# dropout layer
outting = tf.nn.dropout(outting, keep_prob)

# flat into vector
outting = tf.reshape(outting, [-1, nodes * width * width])
nodes = nodes * width * width

# fc layer
# fc1_n = 128
# fc1 = {"weight": tf.Variable(tf.truncated_normal([nodes * width * width, fc1_n], stddev=0.1)),
#       "bias": tf.Variable(tf.truncated_normal([1, fc1_n], stddev=0.1))}

# fc1_z = tf.add(tf.matmul(flatted, fc1["weight"]), fc1["bias"])
# fc1_a = tf.nn.relu(fc1_z)
# print "fc layer build done"

# output layer
out = {"weight": tf.Variable(tf.truncated_normal([nodes, n_classes], stddev=0.1)),
      "bias": tf.Variable(tf.truncated_normal([1, n_classes], stddev=0.1))}

y_ = tf.nn.softmax(tf.add(tf.matmul(outting, out["weight"]), out["bias"]))

# the cost function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_, labels=y))

optm = tf.train.AdamOptimizer().minimize(cost)
corr = tf.equal(tf.argmax(y_, 1), tf.argmax(y, 1))
accr = tf.reduce_mean(tf.cast(corr, tf.float32))
print "cost and optimzer build done"
print "compute graph ready!"

conv1 layer with max pool  build done, output [, 14, 14, 32]
conv2 layer with max pool  build done, output [, 7, 7, 64]
conv3 layer with max pool  build done, output [, 4, 4, 128]
cost and optimzer build done
compute graph ready!


In [58]:
# run!
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
print "init run done"

epochs = 10
batch_size = 128
display_step = 1
total_batch = int(mnist.train.num_examples/batch_size)

writer = tf.summary.FileWriter('/tmp/tf_logs/cnn-mnist', graph=sess.graph)
tf.summary.scalar("accuracy", accr)
summaries = tf.summary.merge_all()

j = 0
for epoch in range(epochs):
    avg_cost = 0.
    print "#Epoch ", epoch
    
    # ITERATION
    for i in range(total_batch):
        j += 1
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feeds = {x: batch_xs, y: batch_ys, keep_prob: 0.8}
        _, percost = sess.run([optm, cost], feed_dict=feeds)
        avg_cost += percost
#         writer.add_summary(summary, j)
    avg_cost = avg_cost / total_batch
    
    # DISPLAY
    if (epoch+1) % display_step == 0:
        print ("Epoch: %03d/%03d cost: %.9f" % (epoch, epochs, avg_cost))
        feeds = {x: batch_xs, y: batch_ys, keep_prob: 1}
        train_acc, = sess.run([accr], feed_dict=feeds)
        print ("TRAIN ACCURACY: %.3f" % (train_acc))
#         writer.add_summary(summary_train, epoch)
        feeds = {x: mnist.test.images, y: mnist.test.labels, keep_prob: 1}
        test_acc = sess.run(accr, feed_dict=feeds)
#         writer.add_summary(test_acc,)
        print ("TEST ACCURACY: %.3f" % (test_acc))
                
print ("OPTIMIZATION FINISHED")

init run done
#Epoch  0
Epoch: 000/010 cost: 1.607381572
TRAIN ACCURACY: 0.969
TEST ACCURACY: 0.972
#Epoch  1
Epoch: 001/010 cost: 1.489170105
TRAIN ACCURACY: 0.977
TEST ACCURACY: 0.983
#Epoch  2
Epoch: 002/010 cost: 1.481952180
TRAIN ACCURACY: 0.992
TEST ACCURACY: 0.987
#Epoch  3
Epoch: 003/010 cost: 1.477369847
TRAIN ACCURACY: 1.000
TEST ACCURACY: 0.989
#Epoch  4
Epoch: 004/010 cost: 1.476044860
TRAIN ACCURACY: 1.000
TEST ACCURACY: 0.989
#Epoch  5
Epoch: 005/010 cost: 1.473342210
TRAIN ACCURACY: 1.000
TEST ACCURACY: 0.990
#Epoch  6
Epoch: 006/010 cost: 1.472104743
TRAIN ACCURACY: 0.992
TEST ACCURACY: 0.990
#Epoch  7
Epoch: 007/010 cost: 1.471542617
TRAIN ACCURACY: 0.984
TEST ACCURACY: 0.992
#Epoch  8
Epoch: 008/010 cost: 1.471017580
TRAIN ACCURACY: 0.992
TEST ACCURACY: 0.991
#Epoch  9
Epoch: 009/010 cost: 1.469936587
TRAIN ACCURACY: 0.992
TEST ACCURACY: 0.992
OPTIMIZATION FINISHED
